## RDP - Stone Soup Experiments
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Continuing experiments - following tutorial #3: Non-Linear Models: *Unscented Kalman Filter*

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all

from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection
from stonesoup.plotter import AnimatedPlotterly

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


## Get Data from BigQuery
* rdp_straight: short, straight flight path
* rdp_extended: longer flight path
* adsb_straight: truth for rdp_straight
* adsb_extended: truth for rdp_extended


## Save/Read  to/from CSV

In [ ]:
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
adsb_file = f'{data_dir}/adsb_straight.csv'
adsb_data = pd.read_csv(adsb_file)
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data = adsb_data.loc[~adsb_data['timestamp'].isna()]
adsb_data['timestamp'] = pd.to_datetime(adsb_data['timestamp'], errors='coerce')
adsb_data['timestamp'] = adsb_data['timestamp'].dt.tz_localize(None)

rdp_file = f'{data_dir}/rdp_straight.csv'
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)

# Matched Plots
matched_csv = f'{data_dir}/rdp_matched.csv'
rdp_matched = pd.read_csv(matched_csv)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()

print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
target_address = 10537421
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = matched_data.loc[(matched_data.target_address==target_address) &
                                (matched_data.close_enough==True)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
# Detections
matched_xy = CSVReaderXY(matched_csv)
matched_polar = CSVReaderPolar(matched_csv)

# ADSB
adsb = ADSBTruthReader.multiple_ground_truth_reader([adsb_file])

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]

timestamps = generate_timestamps(start_time, end_time)
plot_all(dets, adsb, start_time, end_time, plot_type='animated')

## From Tutorial #3

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity

from stonesoup.predictor.kalman import UnscentedKalmanPredictor

# Create :class:`~.UnscentedKalmanUpdater`
from stonesoup.updater.kalman import UnscentedKalmanUpdater
from stonesoup.models.measurement.linear import LinearGaussian

In [ ]:
# Measurement Model
measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[5, 0 ],  
                          [0, 5]])
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_const = 30
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

predictor = UnscentedKalmanPredictor(transition_model)
unscented_updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

# Set Prior for the known track
# using the known starting values
# prior = GaussianState([[90], [-100], [-72], [100]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# using the location of the radar
prior = GaussianState([[0], [50], [0], [50]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)


start_time = min(timestamps)

from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

if "track" in globals():
    del track

track = Track()

for det in dets:
    prediction = predictor.predict(prior, timestamp=det.timestamp)
    # print(det.timestamp)
    hypothesis = SingleHypothesis(prediction, det)   # Group a prediction and measurement
    post = unscented_updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [ ]:
# plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='animated')
plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='static')

Changing the starting prior didn't change the results much at all - so not very sensitive to that! However, the first track point is always oddly somewhere between the radar and the actual first observation. Maybe this will clean up in later iterations of the process.

## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

In [ ]:
# Detections
meas_polar = CSVReaderPolar(matched_csv)

dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]

plot_all(dets, adsb, start_time, end_time, plot_type='animated')

### Kalman Filtering Again

In [ ]:
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.updater.kalman import ExtendedKalmanUpdater
from stonesoup.models.transition.linear import ConstantVelocity

In [ ]:
measurement_model = CartesianToBearingRange(ndim_state=4, 
                                            mapping=(0,2), 
                                            noise_covar=np.diag([np.radians(0.2), 1])
                                            )

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

predictor = UnscentedKalmanPredictor(transition_model)
unscented_updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

from stonesoup.types.state import GaussianState
prior = GaussianState([[90], [-100], [-72], [100]], np.diag([10, 0.5, 10, 0.5]), timestamp=start_time)
# prior = GaussianState([[0], [10], [0], [10]], np.diag([10, 0.5, 10, 0.5]), timestamp=start_time)

from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

if "track" in globals():
    del track
    
track = Track()
dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]

for measurement in dets:
    prediction = predictor.predict(prior, timestamp=measurement.timestamp)
    hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
    post = unscented_updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [ ]:
# plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='animated')
plot_all(dets, adsb, start_time, end_time, q_const, track, plot_type='static')